In [ ]:
!apt-get update
!apt install -y chromium-chromedriver
!pip install selenium gradio

import gradio as gr
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from PIL import Image
import io

In [ ]:
def browse_and_screenshot(url):
    # 1. Setup Chrome Options
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--window-size=1280,720')

    # 2. Initialize Driver
    driver = webdriver.Chrome(options=chrome_options)
    
    try:
        # 3. Navigate to URL
        if not url.startswith("http"):
            url = "https://" + url
        driver.get(url)
        
        # 4. Capture Screenshot
        screenshot_bytes = driver.get_screenshot_as_png()
        img = Image.open(io.BytesIO(screenshot_bytes))
        title = driver.title
        
        driver.quit()
        return img, f"Successfully loaded: {title}"
    except Exception as e:
        if 'driver' in locals(): driver.quit()
        return None, f"Error: {str(e)}"

# 3. Build Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# Colab Chromium Browser UI")
    with gr.Row():
        url_input = gr.Textbox(label="Enter Website URL", placeholder="google.com")
        btn = gr.Button("Go")
    
    status_text = gr.Textbox(label="Status")
    output_image = gr.Image(label="Browser View")

    btn.click(fn=browse_and_screenshot, inputs=url_input, outputs=[output_image, status_text])

# 4. Launch with Public Link
demo.launch(share=True)